In [13]:
import numpy as np
import pandas as pd

In [14]:
import scipy.io

In [15]:
data = scipy.io.loadmat('/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat') 

In [16]:
X = data['X']
y = data['y']
y_flat = data['y'].flatten()

In [17]:
y.mean()

0.14601769911504425

In [18]:
X.max(axis=0)

array([ 83. ,   1. , 780. , 176. , 188. , 524. , 509. , 381. , 205. ,
       169. ,  88. , 156. ,  88. ,  24. ,   0. , 100. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. ,  76. , 132. ,  92. ,  36. ,  56. ,
        76. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,  92. , 116. ,
       132. ,  64. ,  44. ,  92. ,   1. ,   1. ,   1. ,   1. ,   1. ,
         1. , 136. ,  80. ,  80. ,  84. ,  32. ,  92. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. ,  88. , 148. ,  92. ,  44. ,   0. ,
        96. ,   0. ,   1. ,   1. ,   1. ,   1. ,   1. ,  88. , 128. ,
       120. ,  44. ,  44. ,  80. ,   1. ,   1. ,   0. ,   1. ,   1. ,
         1. , 140. , 216. , 116. ,  80. ,  28. , 116. ,   1. ,   1. ,
         1. ,   1. ,   1. ,   1. , 132. , 216. , 108. ,  72. ,  32. ,
        96. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,  96. , 132. ,
       112. ,  76. ,  36. ,  88. ,   1. ,   1. ,   1. ,   1. ,   1. ,
         1. ,  88. ,  92. , 124. ,  24. ,  40. ,  60. ,   1. ,   1. ,
         0. ,   0. ,

In [19]:
X.min(axis=0)

array([   0. ,    0. ,  105. ,    6. ,   55. ,    0. ,  232. ,  108. ,
          0. , -172. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
      

In [20]:
from anomaly.base import AnomalyDetector

In [21]:
from river import metrics

In [22]:
from models.rs_hash import RSHash

In [23]:
from river.stream import iter_array

# Test Dico

In [24]:
model = RSHash(X.min(axis=0),X.max(axis=0))
ROCAUC = metrics.ROCAUC()
for xi, yi in iter_array(X,y_flat):
    
    model = model.learn_one(xi)
    anomaly_score = model.score_one(xi)
    
    ROCAUC.update(yi,anomaly_score)
print(ROCAUC)

ROCAUC: 28.29%


# OLD VERSION

In [12]:
model = RSHash(X.min(axis=0),X.max(axis=0))
ROCAUC = metrics.ROCAUC()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    model = model.learn_one(x_array)           #pas dans le bon sens pour river
    anomaly_score = model.score_one(x_array)   #A FAIRE: il faut que anomaly_score soit entre 1 et 0 pour river
    
#    print(y_array,anomaly_score)
    ROCAUC.update(int(y_array[0]),anomaly_score)
print(ROCAUC)

(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)
(274,)

ROCAUC: 28.52%
le modèle ne semble rien apprendre !

In [12]:
from pysad.evaluation import AUROCMetric

In [14]:
model = RSHash(X.min(axis=0),X.max(axis=0))
ROCAUC = AUROCMetric()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    model = model.learn_one(x_array)           #pas dans le bon sens pour river
    anomaly_score = model.score_one(x_array)   #A FAIRE: il faut que anomaly_score soit entre 1 et 0 pour river
    
#    print(y_array,anomaly_score)
    ROCAUC.update(int(y_array[0]),anomaly_score)
print(ROCAUC.get())

0.24945046318103312


In [47]:
from pysad.transform.probability_calibration import ConformalProbabilityCalibrator

In [48]:
model = RSHash(X.min(axis=0),X.max(axis=0))
calibrator = ConformalProbabilityCalibrator(windowed=True, window_size=300)
ROCAUC = AUROCMetric()
for xi, yi in iter_array(X,y):
    x_array = np.array(list(xi.values()))
    y_array = np.array(list(yi.values()))
    model = model.learn_one(x_array)           #pas dans le bon sens pour river
    anomaly_score = model.score_one(x_array)   #A FAIRE: il faut que anomaly_score soit entre 1 et 0 pour river
    calibrated_score = calibrator.fit_transform(anomaly_score)
#    print(y_array,anomaly_score)
    ROCAUC.update(int(y_array[0]),calibrated_score)
print(ROCAUC.get())

IndexError: tuple index out of range